In [ ]:
!pip install ../input/pytorch-190/torch-1.9.0+cu111-cp37-cp37m-linux_x86_64.whl

In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn import datasets
from sklearn import model_selection
import gc
from fastai.vision.all import *

In [ ]:
path = Path('../input/petfinder-pawpularity-score')

In [ ]:
df_train = pd.read_csv(path/'train.csv')
df_test  = pd.read_csv(path/'test.csv')

In [ ]:
path

In [ ]:
df_train.Id = df_train.Id.map(lambda x:str(path) + '/train/' + x + '.jpg')
df_test.Id = df_test.Id.map(lambda x:str(path) + '/test/' + x + '.jpg')

In [ ]:
ymin = 0.00001
ymax = 100

class scaledSigmoid(nn.Module):
    def forward(self, input):
        return torch.sigmoid(input) * (ymax - ymin) + ymin

class clampedReLU(nn.Module):
    def forward(self, input):
        bottomClamp = input < ymin
        topClamp = input > ymax
        input[bottomClamp,] = ymin
        input[topClamp,] = ymax
        return input


In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

In [ ]:
def petfinder_rmse(input,target,segmoid=True):
    if segmoid == True:
        return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))
    else:
        return 100*torch.sqrt(F.mse_loss(input.flatten(), target))

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model


In [ ]:
    data_with_sudolabel = pd.concat((df_train,df_test),axis=0)
    cols = list(data_with_sudolabel.columns[1:-1])
    data_with_sudolabel['labels'] = -1
    label_list = []
    for row in data_with_sudolabel.iterrows():
        (_,row) = row
        label_list.append([col for col in cols if row[col]==1])
    data_with_sudolabel['labels'] = label_list
    def get_x(r):return r['Id']
    def get_y(r):return r['labels']
    def get_dls(bs,size,df,mult=1):
        dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock), #pass in train DataFrame
                                   splitter=IndexSplitter([1]),
                                   get_x=get_x, #filename/path is in the second column of the DataFrame
                                   get_y=get_y, #label is in the first column of the DataFrame
                                   item_tfms=Resize(224), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()]))
        dls = dblock.dataloaders(data_with_sudolabel,bs=bs)
        dsets = dblock.datasets(data_with_sudolabel)
        return dls,dsets
    dls_psudolabel,dsets = get_dls(bs=32,size=224,df=data_with_sudolabel)

In [ ]:
dls_psudolabel.train.one_batch()[0].shape

In [ ]:
dls_psudolabel.show_batch()

In [ ]:

model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=12)
learn = Learner(dls_psudolabel,model)
learn.unfreeze()
learn.fit_one_cycle(3,lr_max=slice(1e-4,1e-3),cbs=[MixUp(0.2)])

In [ ]:
df_train['norm_score'] = df_train['Pawpularity']/100
df_train['norm_score']

In [ ]:
def get_dls(bs,mult=1):
    train_df_f = df_train.copy()
    def get_x(r):return r['Id']
    def get_y(r):return r['norm_score']
    dblock = DataBlock(blocks=(ImageBlock, RegressionBlock), #pass in train DataFrame
                                   splitter=IndexSplitter([1]),
                                   get_x=get_x, #filename/path is in the second column of the DataFrame
                                   get_y=get_y, #label is in the first column of the DataFrame
                                   item_tfms=Resize(224), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()]))
    dls = dblock.dataloaders(train_df_f,bs=bs)
    dsets = dblock.datasets(train_df_f)
    return dls,dsets

In [ ]:
dls, _ = get_dls(16)
learn.model.head = nn.Linear(1536,1,bias=False)
learn.unfreeze()
learn.dls = dls
learn.fit_one_cycle(6,lr_max=slice(2e-4),cbs=[MixUp(0.2)])

In [ ]:
test_dl = dls.test_dl(df_test)
(preds,tta_preds), _ = learn.tta(dl=test_dl, n=1, beta=None)

In [ ]:
sample_df = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
sample_df['Pawpularity'] = preds*100
sample_df.to_csv('submission.csv',index=False)

In [ ]:
sample_df